# MCP Server Guide for Biomni

MCP servers connect external APIs to Biomni. This guide shows how to use the built-in servers and create your own.

## PubMed Implementation
The actual PubMed server code shows the MCP pattern:

```python
from mcp.server.fastmcp import FastMCP
from pydantic import BaseModel
from Bio import Entrez

mcp = FastMCP("pubmed-live")

class Article(BaseModel):
    pmid: str
    title: str | None = None
    journal: str | None = None
    year: str | None = None

@mcp.tool()
def search_pubmed(query: str, max_results: int = 10) -> list[Article]:
    ids = Entrez.read(Entrez.esearch(db="pubmed", term=query, retmax=max_results))["IdList"]
    summaries = Entrez.read(Entrez.esummary(db="pubmed", id=",".join(ids))) if ids else []
    
    articles = []
    for s in summaries:
        articles.append(Article(
            pmid=s["Id"],
            title=s["Title"],
            journal=s.get("FullJournalName", ""),
            year=s.get("PubDate", "")[:4]
        ))
    return articles

if __name__ == "__main__":
    mcp.run()
```

The `mcp_config.yaml` file maps Python functions to Biomni tools:

```yaml
mcp_servers:
  pubmed:
    command: ["python", "-m", "biomni.tool.mcp_tools.pubmed_mcp"]
    enabled: true
    tools:
      - biomni_name: search_pubmed
        description: "Search PubMed"
        parameters:
          query: {type: str, required: true}
          max_results: {type: int, required: false, default: 10}
```

## Setup  
First, initialize Biomni and load the MCP configuration:

In [ ]:
from biomni.agent import A1

# Optional: Set email for PubMed
# import os
# os.environ.setdefault("NCBI_EMAIL", "Your email address")

agent = A1()
agent.add_mcp(config_path="./mcp_config.yaml")

Checking and downloading missing data lake files...


## Using PubMed MCP

This example forces Biomni to use specific MCP tools:

In [2]:
result = agent.go(
    "Search PubMed for BRCA1 and list two paper titles with PMIDs. "
    "Must use pubmed_mcp, search_pubmed and get_article_abstract. "
    "If pubmed_mcp fails, end this entire task."
)
print(result)

Using prompt-based retrieval with the agent's LLM
================================ Human Message =================================

Search PubMed for BRCA1 and list two paper titles with PMIDs. Must use pubmed_mcp, search_pubmed and get_article_abstract. If pubmed_mcp fails, end this entire task.
================================== Ai Message ==================================

I'll help you search PubMed for BRCA1 papers and retrieve their titles with PMIDs. Let me create a plan first.

**Plan:**
1. [ ] Use the search_pubmed function from mcp_servers.pubmed to search for BRCA1 papers
2. [ ] Extract PMIDs from the search results
3. [ ] Use get_article_abstract function to get detailed information for two papers
4. [ ] Display the paper titles with their PMIDs

Let me start by searching PubMed for BRCA1 papers:

<execute>
from mcp_servers.pubmed import search_pubmed, get_article_abstract

print("Step 1: Searching PubMed for BRCA1 papers...")
try:
    search_results = search_pubmed(query=

## ToolUniverse Implementation

This uses [ToolUniverse MCP](https://github.com/mims-harvard/ToolUniverse) server with all 214 functions derived from FDA, OpenTargets, and HPO.  
Natural language query that automatically selects the right FDA tools:

The actual ToolUniverse server code shows the MCP pattern:

```python
from fastmcp import FastMCP
from tooluniverse.execute_function import ToolUniverse

mcp = FastMCP("ToolUniverse MCP", stateless_http=True)
engine = ToolUniverse()
engine.load_tools()


@mcp.tool()
def FDA_get_active_ingredient_info_by_drug_name(drug_name: str, limit: int, skip: int) -> dict:
    return engine.run_one_function(
        {
            "name": "FDA_get_active_ingredient_info_by_drug_name",
            "arguments": {"drug_name": drug_name, "limit": limit, "skip": skip},
        }
    )

if __name__ == "__main__":
    mcp.run()
```

The `mcp_config.yaml` file maps Python functions to Biomni tools:

```yaml
mcp_servers:
  tooluniverse:
    command: ["python", "-m", "biomni.tool.mcp_tools.tooluniverse_mcp"]
    enabled: true
    tools:
      - biomni_name: FDA_get_active_ingredient_info_by_drug_name
        description: "Get FDA active ingredient information by drug name"
        parameters:
          drug_name: {type: str, required: true, description: "Name of the drug to search for"}
          limit:     {type: int, required: true, description: "Maximum number of results to return"}
          skip:      {type: int, required: true, description: "Number of results to skip"}
```

## Using ToolUniverse MCP

This example forces Biomni to use specific MCP tools:

In [3]:
result = agent.go("I need to find FDA active ingredient information for the drug 'ibuprofen'.")
print(result)

Using prompt-based retrieval with the agent's LLM
================================ Human Message =================================

I need to find FDA active ingredient information for the drug 'ibuprofen'.
================================== Ai Message ==================================

I'll help you find FDA active ingredient information for ibuprofen. Let me create a plan to accomplish this task.

**Plan:**
1. [ ] Use the FDA_get_active_ingredient_info_by_drug_name function to search for ibuprofen
2. [ ] Display and analyze the results
3. [ ] Provide a comprehensive summary of the FDA active ingredient information

Let me start by executing the search:

<execute>
# Import the FDA function
from mcp_servers.tooluniverse import FDA_get_active_ingredient_info_by_drug_name

# Search for ibuprofen active ingredient information
print("Searching FDA database for ibuprofen active ingredient information...")
result = FDA_get_active_ingredient_info_by_drug_name(drug_name="ibuprofen", limit=10)

## Tool Configuration Files

The YAML configuration files used to activate these tools are located in:
`biomni/tool/mcp_tools/mcp_configs/tooluniverse`

They are organized into three categories based on data source:
---

#### **FDA Category (12 files)**
- `fda_drug_admin.yaml`: 20 tools  
- `fda_safety.yaml`: 29 tools  
- `fda_other.yaml`: 25 tools  
- `fda_populations.yaml`: 12 tools  
- `fda_clinical.yaml`: 10 tools  
- `fda_interactions.yaml`: 10 tools  
- `fda_regulatory.yaml`: 10 tools  
- `fda_composition.yaml`: 9 tools  
- `fda_patient_info.yaml`: 9 tools
- `fda_abuse.yaml`: 8 tools  
- `fda_pharmacology.yaml`: 8 tools  
- `fda_toxicology.yaml`: 6 tools  

---

#### **OpenTargets Category (7 files)**
- `ot_target_biology.yaml`: 10 tools  
- `ot_other.yaml`: 10 tools  
- `ot_search.yaml`: 8 tools  
- `ot_disease_links.yaml`: 7 tools  
- `ot_disease_info.yaml`: 6 tools  
- `ot_drug_safety.yaml`: 6 tools  
- `ot_research.yaml`: 6 tools  

---

#### **HPO Category (1 file)**
- `hpo_phenotype.yaml`: 5 tools